In [ ]:
from tqdm import tqdm
from dask import delayed, compute
from dask.distributed import Client

message = """Professor Agar says, 'Come get it, debug your code like a rockstar! 💻🎤
When your loops are infinite, just sing Aaron Carter's 'Come Get It' and the bugs will flee! 🐛🏃‍♂️
Remember, coding is like dancing—you gotta keep the rhythm with proper indentation. 🕺🐍
And if you ever feel stuck, just shout, 'Come get it!' and the solution will come running. 🏃‍♀️💡
Now go out there and slay that code, Python warriors! 🐍⚔️'
Watch the music video here: https://www.youtube.com/watch?v=y0p3jn7ODuc"""

key = "antiferroelectric53^"


def xor_encrypt(message, key):
    """Encrypts a message using XOR encryption with the given key."""
    encrypted_message = bytearray()
    key_length = len(key)
    for i in range(len(message)):
        encrypted_message.append(message[i] ^ key[i % key_length])

    with open("encrypted_message.txt", "wb") as f:
        f.write(encrypted_message)

    return encrypted_message


encrypted_message = xor_encrypt(message.encode("utf-8"), key.encode("utf-8"))


def xor_decrypt(encrypted_message, key):
    """Decrypts an XOR-encrypted message using the given key."""
    decrypted_message = bytearray()
    key = key.encode("utf-8")  # Convert key to bytes
    key_length = len(key)
    for i in range(len(encrypted_message)):
        decrypted_message.append(encrypted_message[i] ^ key[i % key_length])
    return decrypted_message.decode("utf-8", errors="ignore")


def generate_passwords(wordlist, symbols):
    """Generates all possible passwords based on Professor Agar's rules."""
    passwords = []
    for word in wordlist:
        # Add the word as-is and with the first letter capitalized
        variations = [word, word.capitalize()]
        for w in variations:
            # Append up to 2 numbers
            for num in range(100):
                num_str = f"{num}"
                # Append a symbol
                for symbol in symbols:
                    password = f"{w}{num_str}{symbol}"
                    passwords.append(password)
    return passwords


def crack_password_chunk(encrypted_message, passwords_chunk):
    """Cracks a chunk of passwords."""
    for password in passwords_chunk:
        key = password.encode("utf-8")
        decrypted_message = xor_decrypt(encrypted_message, key)
        # Check if the decrypted message contains readable text
        if all(
            word in decrypted_message.lower()
            for word in ["if", "the", "and", "https://"]
        ):  # Example check
            return password, decrypted_message
    return None, None


def crack_password(encrypted_message, wordlist, symbols):
    """Cracks the password by brute-forcing all possible combinations in parallel using Dask."""
    passwords = generate_passwords(wordlist, symbols)
    chunk_size = len(passwords) // (
        len(Client().ncores()) * 4
    )  # Dynamically determine chunk size
    password_chunks = [
        passwords[i : i + chunk_size] for i in range(0, len(passwords), chunk_size)
    ]

    # Use Dask to parallelize the cracking process
    delayed_results = [
        delayed(crack_password_chunk)(encrypted_message, chunk)
        for chunk in password_chunks
    ]

    # Compute results in parallel
    results = compute(*delayed_results)

    # Check results for the correct password
    for result in results:
        password, decrypted_message = result
        if password:
            print(f"Password found: {password}")
            print(f"Decrypted message: {decrypted_message}")
            return password
    return None


# Load the encrypted message and wordlist
with open("encrypted_message.txt", "rb") as f:
    encrypted_message = f.read()

with open("words.txt", "r") as f:
    wordlist = [line.strip() for line in f.readlines()]

symbols = ["!", "@", "#", "$", "%", "^", "&", "*"]

# Initialize Dask client to use all cores
client = Client()

# Crack the password
password = crack_password(encrypted_message, wordlist, symbols)

if password:
    print(f"Success! The password is: {password}")
    # Use the password to reveal the song
    # (This part would involve further decryption or accessing a system)
else:
    print("Failed to crack the password.")